In [34]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

from helper import *
from dataset_class import *
from unet import UNet

In [2]:
# Load images from folders A and B
folder_A = 'trainval/A'
folder_B = 'trainval/B'
folder_label = 'trainval/label'

In [3]:
device = ("cuda" if torch.cuda.is_available() else "cpu") # Use GPU or CPU for training

In [4]:
print(device)

cuda


In [5]:
images_A = load_images_from_folder(folder_A, is_gray=False)

In [6]:
images_B = load_images_from_folder(folder_B, is_gray=False)

In [7]:
labels = load_images_from_folder(folder_label, is_gray = False)

In [8]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [9]:
dataset = ChangeDetectionDataset(images_A, images_B, labels, transform=transform)

In [10]:
del images_A, images_B, labels

In [11]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

In [12]:
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

In [13]:
del dataset

In [14]:
batch_size = 16

In [15]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [16]:
del train_dataset,val_dataset

In [35]:
model = UNet(n_channels=6, n_classes=1)
model = model.to(memory_format=torch.channels_last)

In [36]:
model.to(device=device)

UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [37]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0005)
grad_scaler = torch.cuda.amp.GradScaler(enabled=True)

In [38]:
# Training loop
num_epochs = 3

total_true = []
total_pred = []

for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    total_intersection = 0
    total_union = 0
    
    for batch_idx, (data_A, data_B, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data_A = data_A.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        data_B = data_B.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
        target = target.to(device=device, dtype=torch.long)

        with torch.autocast(device if device != 'mps' else 'cpu', enabled=True):
            x = torch.cat([data_A, data_A], dim=1)
            output = model(x)
            print(output)
            loss = criterion(output, target)

        optimizer.zero_grad(set_to_none=True)
        grad_scaler.scale(loss).backward()
        grad_scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        grad_scaler.step(optimizer)
        grad_scaler.update()
        
        total_train_loss += loss.item()

        # print("Batch index train:",batch_idx)


    # Validation
    model.eval()
    total_val_loss = 0
    total_val_intersection = 0
    total_val_union = 0
    
    with torch.no_grad():
        for batch_idx, (data_A, data_B, target) in enumerate(val_loader):
            data_A = data_A.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            data_B = data_B.to(device=device, dtype=torch.float32, memory_format=torch.channels_last)
            target = target.to(device=device, dtype=torch.long)
            with torch.autocast(device.type if device.type != 'mps' else 'cpu', enabled=True):
                x = torch.cat([data_A, data_A], dim=1)
                output = model(x)
            val_loss = criterion(output, target)
            total_val_loss += val_loss.item()

            total_pred.append(output)
            total_true.append(target)



    print(f'Epoch {epoch+1}, Train Loss: {total_train_loss/len(train_loader)}'
          f'Validation Loss: {total_val_loss/len(val_loader)}')


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 9.82 GiB is allocated by PyTorch, and 27.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)